**Bank Marketing 데이터셋**

고객의 인구통계적 정보 및 이전 마케팅 이력 데이터를 바탕으로 "이 사람이 정기예금 상품에 가입할 것인가?" 를 예측하는 것이 목적입니다.

제공된 학습용 데이터(bank_train.csv)를 이용하여 정기예금 상품에 가입 여부를 예측하는 모델을 개발하고, 개발한 모델에 기반하여 평가용 데이터(bank_test.csv)에 적용하여 얻은 정기예금 상품에 가입 여부 예측 확률을 아래 [제출형식]에 따라 csv 파일로 생성하여 제출하시오.
- 예측 결과는 ROC-AUC 평가지표에 따라 평가함
- 성능이 우수한 예측 모델을 구축하기 위해서는 데이터 정제, Feature Engineering, 하이퍼 파라미터(hyper parameter) 최적화, 모델 비교 등이 필요할 수 있음. 다만, 과적합에 유의하여야 함


[[제출 형식]]
- 가. CSV 파일명: result.csv(파일명에 디렉토리/폴더 지정불가
- 나. 예측 성별 칼럼명 : pred
- 다. 제출 칼럼 개수 : pred 칼럼 1개
- 라. 평가용 데이터 개수와 예측 결과 데이터 개수 일치 : 1,221개

[[제공 데이터]]
- 데이터 목록
- bank_train.csv : 학습용 데이터, 3,300개
- bank_test.csv : 평가용 데이터, 1,221개
- 평가용 데이터는 'term_deposit' 칼럼 미제공

[[ CSV 파일 제출 방법 ]]
- CSV 파일명 : result.csv
- 정기예금 가입이 yes일 확률 칼럼명 : pred
- 제출 CSV 파일 형식 예시
        pred
        0.65564633
        0.98456441
        0.4364433
        0.20123542
        0.82009154

* 소수점 아래자릿수는 별도 지정하지 않음
* pred 칼럼 데이터 수는 1,221

In [ ]:
import pandas as pd

path = 'https://raw.githubusercontent.com/Soyoung-Yoon/bigdata/main/'
train = pd.read_csv(path + 'bank_train.csv')
test = pd.read_csv(path + 'bank_test.csv')

# 실행코드
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

# 데이터 확인
# print(train.isna().sum().to_frame().T)
# print(test.isna().sum().to_frame().T)
# print(train.head(3), train.shape, sep="\n") # (3300, 14)
# print(test.head(3), test.shape, sep="\n")   # (1221, 13)

X = train.drop(columns=['term_deposit'])
Y = train['term_deposit']
X_all = pd.concat([X, test])

# 범주형 컬럼에 대한 Encoder 전처리
columns_obj = X_all.select_dtypes(include='object').columns
for col in columns_obj:
    X_all[col] = LabelEncoder().fit_transform(X_all[col])
# print(X_all.head(3))

# Scaling적용
temp1 = StandardScaler().fit_transform(X_all)
X_all = pd.DataFrame(temp1, columns=X_all.columns)

# 전처리 전의 데이터로 분리
X = X_all.iloc[:3300, :]
X_submission = X_all.iloc[3300:, :]
# print(X.shape, Y.shape, X_submission.shape) # (3300, 13) (3300,) (1221, 13)

# 훈련용, 테스트용 데이터 분리
temp = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=1234)
x_train, x_test, y_train, y_test = temp
# print(x_train.shape, x_test.shape, y_train.shape, y_test.shape) # (2310, 13) (990, 13) (2310,) (990,)

# 성능평가 사용자 함수
def get_scores(model, x_train, x_test, y_train, y_test):
    A = model.score(x_train, y_train)
    B = model.score(x_test, y_test)
    y_proba1 = model.predict_proba(x_train)[:, 1]
    y_proba2 = model.predict_proba(x_test)[:, 1]
    C = roc_auc_score(y_train, y_proba1)
    D = roc_auc_score(y_test, y_proba2)
    return f"acc: {A:.4f}, {B:.4f}, AUC: {C:.4f}, {D:.4f}"

# 모델적합

model1 = LogisticRegression().fit(x_train, y_train)
# print(get_scores(model1, x_train, x_test, y_train, y_test)) # acc: 0.8841, 0.8818, AUC: 0.6949, 0.7035

# model2 = DecisionTreeClassifier(random_state=1234).fit(x_train, y_train)
# print(get_scores(model2, x_train, x_test, y_train, y_test)) # acc: 1.0000, 0.8091, AUC: 1.0000, 0.5430
# print(model2.get_depth()) # 26
# for d in range(3, 13):
#     model2 = DecisionTreeClassifier(max_depth=d, random_state=1234).fit(x_train, y_train)
#     print(f"{d}", get_scores(model2, x_train, x_test, y_train, y_test))
# 4 acc: 0.8973, 0.8894, AUC: 0.7126, 0.6856
model2 = DecisionTreeClassifier(max_depth=4, random_state=1234).fit(x_train, y_train)
# print(get_scores(model2, x_train, x_test, y_train, y_test)) # acc: 0.8973, 0.8894, AUC: 0.7126, 0.6856

# model3 = RandomForestClassifier(random_state=0).fit(x_train, y_train)
# print(get_scores(model3, x_train, x_test, y_train, y_test)) # acc: 1.0000, 0.8828, AUC: 1.0000, 0.7255
# for d in range(1, 16):
#     model3 = RandomForestClassifier(max_depth=d, random_state=1342).fit(x_train, y_train)
#     print(d, get_scores(model3, x_train, x_test, y_train, y_test))
# 3 acc: 0.8857, 0.8848, AUC: 0.7341, 0.7362
model3 = RandomForestClassifier(max_depth=2, random_state=1234).fit(x_train, y_train)
# print(get_scores(model3, x_train, x_test, y_train, y_test)) # acc: 0.8857, 0.8848, AUC: 0.7341, 0.7362

model4 = AdaBoostClassifier(random_state=1234).fit(x_train, y_train)
# print(get_scores(model4, x_train, x_test, y_train, y_test))

model5 = GradientBoostingClassifier(random_state=1234).fit(x_train, y_train)
# print(get_scores(model5, x_train, x_test, y_train, y_test))

# 모델선택, 예측값 산출
final_model = model3
y_pred = final_model.predict(X_submission)

#결과 저장 및 확인용 코드
pd.DataFrame({'pred': y_pred}).to_csv('result.csv', index=False)
# temp = pd.read_csv('result.csv')
# print(temp.head())
# print(temp.value_counts())
# print(Y[:1221].value_counts())

1 acc: 0.8848, 0.8848, AUC: 0.7002, 0.6985
2 acc: 0.8848, 0.8848, AUC: 0.7123, 0.7237
3 acc: 0.8856, 0.8864, AUC: 0.7297, 0.7144
4 acc: 0.8898, 0.8864, AUC: 0.7655, 0.7243
5 acc: 0.8989, 0.8879, AUC: 0.8051, 0.7241
6 acc: 0.9038, 0.8879, AUC: 0.8508, 0.7240
7 acc: 0.9064, 0.8879, AUC: 0.8975, 0.7129
8 acc: 0.9114, 0.8924, AUC: 0.9269, 0.7192
9 acc: 0.9170, 0.8894, AUC: 0.9567, 0.7124
10 acc: 0.9239, 0.8894, AUC: 0.9793, 0.6951
11 acc: 0.9330, 0.8909, AUC: 0.9904, 0.7084
12 acc: 0.9420, 0.8894, AUC: 0.9964, 0.7000
13 acc: 0.9515, 0.8879, AUC: 0.9989, 0.6882
14 acc: 0.9602, 0.8864, AUC: 0.9996, 0.6947
15 acc: 0.9686, 0.8864, AUC: 0.9999, 0.6904
acc: 0.8848, 0.8848, AUC: 0.7142, 0.7123
